In [1]:
import numpy as np
import pandas as pd
import math
import scipy
import sys
from tqdm import tqdm
import functools
import networkx as nx
import itertools

import portion as P
import re
import datetime
import queue

# interval = P.empty()
# for i, j in [(0, 1), (2, 4), (6, 7), (3, 5)]:
#     interval = interval | P.closed(i, j)
# print(interval)

In [2]:
scipy.signal.correlate2d(np.array([
    [1,2,0],
    [1,0,0],
    [1,0,0]]), np.array([[1,1,1],
                         [1,1,0],
                         [1,0,0]]), mode='same')

array([[1, 4, 2],
       [4, 5, 2],
       [2, 2, 0]])

In [82]:
row = np.zeros((1,9))
row[0,0] = 8
row[0,8] = 8

grid = np.zeros((2,9))
grid[0,1] = grid[0,8] = 8
grid[1,0] = grid[1,8] = 1
grid


array([[0., 8., 0., 0., 0., 0., 0., 0., 8.],
       [1., 0., 0., 0., 0., 0., 0., 0., 1.]])

In [103]:
grid[1:2,:] = row

In [2]:
# ##all rocks

# np.ones((1,4))

# tmp = np.ones((3,3))
# tmp[0,0] = tmp[2,2] = tmp[0,2] = tmp[2,0] = 0
# tmp

# tmp = np.zeros((3,3))
# tmp[2,:] = 1
# tmp[:,2] = 1
# tmp

# np.ones((4,1))

# np.ones((2,2))

In [622]:
np.ones((1,9)).tolist()

[[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]]

In [118]:
t = []
t.append((np.ones((1,9)).tolist(),2))
t.append((np.ones((1,9)).tolist(),4))

(np.ones((1,9)).tolist(),3) in t

# if ((np.ones((1,9)),2) == (np.ones((1,9)),4)).all():
#     print('test')
str(t)

'[([[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]], 2), ([[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]], 4)]'

In [104]:
class Grid():
    grid = np.zeros((1,9)) + 8
    directions = None
    shapes = None
    drops = 0
    shape_num = -1
    removed = 0
    history = None
    history_skip_by = None

    row = np.zeros((1,9))
    row[0,0] = 8
    row[0,8] = 8

    def __init__(self, direction_str) -> None:
        self.directions = []
        self.history = {}
        self.history_skip_by = {}

        for i in list(direction_str):
            self.directions.append(-1 if i == '<' else 1)
        
        self.shapes = [np.ones((1,4))]

        tmp = np.ones((3,3))
        tmp[0,0] = tmp[2,2] = tmp[0,2] = tmp[2,0] = 0
        self.shapes.append(tmp)

        tmp = np.zeros((3,3))
        tmp[2,:] = 1
        tmp[:,2] = 1
        self.shapes.append(tmp)
        self.shapes.append(np.ones((4,1)))
        self.shapes.append(np.ones((2,2)))
        

    def add_rows_to_top(self, num):
        tmp = self.row.repeat(num, axis=0).reshape((num,9))
        self.grid = np.concatenate((tmp, self.grid), axis=0)

    def find_first_non_empty_row(self):
        for i,x in enumerate(self.grid.dot(np.ones((1,9)).T)):
            if x != 16:
                return i
        #if no empty rows, add one and return 1
        self.add_rows_to_top(1)
        return 1

    def __repr__(self) -> str:
        ret = ''
        for i in range(self.grid.shape[0]):
            for j in range(self.grid.shape[1]):
                if self.grid[i,j] == 0:
                    ret += '.'
                elif self.grid[i,j] == 8:
                    ret += '#'
                elif self.grid[i,j] == 1:
                    ret += '@'
            ret += '\n'
        return ret
    def next_move(self):
        el = self.directions.pop(0)
        # print(f'direction: {el}')
        self.directions.append(el)
        return el
    
    def get_shape(self):
        self.shape_num = (self.shape_num+1) % len(self.shapes)
        return self.shapes[self.shape_num]

    def peek_shape(self):
        return self.shapes[(self.shape_num+1) % len(self.shapes)]

    def check_history_cache(self, current_shape):
        current_state = str((current_shape,self.grid, self.directions))
        if current_state in self.history:
            if current_state not in self.history_skip_by:
                prev_height, prev_drops = self.history[current_state]
                self.history_skip_by[current_state] = (self.height() - prev_height, self.drops - prev_drops)

            # print(f'found repeat! skipping by {self.history_skip_by[current_state]}')
            return self.history_skip_by[current_state]

        else:
            self.history[current_state] = (self.height(),self.drops)
        return 0
    

    def drop(self):
        def can_move_down(shape_top_loc, left_edge, shape):
            shape_height = shape.shape[0]
            shape_width = shape.shape[1]
            shape_top_loc_next = shape_top_loc+1

            # print(left_edge)
            # print(self.grid[shape_top_loc_next:shape_top_loc_next+shape_height, left_edge:left_edge+shape_width])
            # print(shape)

            if sum(sum(self.grid[shape_top_loc_next:shape_top_loc_next+shape_height, left_edge:left_edge+shape_width] * shape)) == 0:
                return True
            return False
        def slide(shape_top_loc, left_edge, shape):
            #returns new left_edge
            shape_height = shape.shape[0]
            shape_width = shape.shape[1]
            new_left_edge = left_edge + self.next_move()
            # print(f'{left_edge=}, {new_left_edge=}')
            
            if sum(sum(self.grid[shape_top_loc:shape_top_loc+shape_height, new_left_edge:new_left_edge+shape_width] * shape)) == 0:
                return new_left_edge
            return left_edge

        shape = self.get_shape()
        # print(f'drop:', shape)
        left_edge = 3
        first_non_empty_line = self.find_first_non_empty_row()
        space_needed = shape.shape[0] + 3
        # print(f'{shape=} {first_non_empty_line=} {space_needed=}')
        
        if first_non_empty_line < space_needed:
            self.add_rows_to_top(space_needed - first_non_empty_line)
            first_non_empty_line = self.find_first_non_empty_row()


        shape_top_loc = first_non_empty_line - space_needed
        # print(f'{left_edge=}')
        left_edge = slide(shape_top_loc, left_edge, shape)
        # print(f'{left_edge=}')
        # print(shape_top_loc)
        while can_move_down(shape_top_loc, left_edge, shape):
            shape_top_loc += 1
            left_edge = slide(shape_top_loc, left_edge, shape)
            # print(f'{left_edge=}')
        
        #place the rock in place
        shape_height = shape.shape[0]
        shape_width = shape.shape[1]

        orig_grid = self.grid[shape_top_loc:shape_top_loc+shape_height, left_edge:left_edge+shape_width]
        orig_grid += shape
        orig_grid[orig_grid > 0] = 1

        self.grid[shape_top_loc:shape_top_loc+shape_height, left_edge:left_edge+shape_width] = orig_grid
        self.drops += 1
        self.trim_excess_bottom()
    
    def height(self):
        return self.removed + self.grid.shape[0] - self.find_first_non_empty_row() -1

    def trim_excess_bottom(self):
        #dijkstra's for seeing what's reachable
        def find_neighbors(i, j, grid):
            #checks only top, down, left, right
            ret = []
            if self.grid[i-1,j] == 0:
                ret.append((i-1,j))
            if self.grid[i+1,j] == 0:
                ret.append((i+1,j))
            if self.grid[i,j-1] == 0:
                ret.append((i,j-1))
            if self.grid[i,j+1] == 0:
                ret.append((i,j+1))
            return ret

        grid = self.grid.copy()

        current_set = [(grid.shape[0]-2, i) for i in range(1,8)]
        visited_set = set()

        while current_set:
            # print(len(current_set),len(visited_set))
            i,j = current_set.pop(0)
            grid[i,j] = 2
            visited_set = visited_set | {(i,j)}
            neighbors = find_neighbors(i, j, grid)
            for n in neighbors:
                if (n not in current_set) and (n not in visited_set):
                    current_set.append(n)
        # print(grid)

        row = 0
        while min(grid[row,:]) == 0:
            row += 1
        row
        
        if row:
            self.removed += self.grid.shape[0]-1 - row
            self.grid = np.delete(self.grid, slice(row,self.grid.shape[0]-1), axis=0)




    def run(self, rounds):
        #for second part
        i = 0
        extra_height = 0
        while i < rounds:
            # print(i)
            # if not (i % 1000):
                # print(i)
            
            ## add state to self.history
            if data := self.check_history_cache(self.peek_shape()):
                increase_by_height, increase_drops = data
                # print(data)
                if increase_drops + i < rounds:
                    repeat_num = (rounds - i) // increase_drops
                    print(f'skipped {repeat_num*increase_drops} iterations')
                    i += repeat_num*increase_drops
                    extra_height =+ repeat_num*increase_by_height
                    
            data = self.drop()
            i += 1
            
            self.trim_excess_bottom()
        print(f'{i=}')
        return self.height() + extra_height
        



In [115]:
# input = '>>><<><>><<<>><>>><<<>>><<<><<<>><>><<>>'
input = '>>><>><<<><<<>><>><>><>>><><<<><<<<><>>>><>>>><<<<>>><<<<>><<<<>>><<<>>><<<>><<>>><>>><<>><<<<>>>><<<><<<><>>><<>>>><<<>>><<><<<>>>><<<><>><<<>>>><<>>>><<<<>><<<><<<<><>>>><<>>>><<>>>><<<<><<<<>>><><<>>><>>><<>>><<<<>>>><<<>><<<<>>>><<><<>>>><<<>><<><<<>>>><<>>>><<<<>>>><<<><<>>><<<>><<<<>><<>>>><<<>>>><<>>>><<>>>><<<<>><>>>><<<<><<>>>><>><>>><<>>>><<<<>>><>><><<<><<>><<<><<>>><<<><<<><<<<><<<>>><><<<<>><<<>>><<><<<<>>><<<<>>>><<<>>><<<>>>><<><<<<>><><<>>>><<<><<>>><<>><<<<>><<<>><>><>><>>><<<>>>><>>>><<><<<>>><>>>><<<>>>><>><<>><>>>><<<>>><<><<><<<<>>><<>><>>><<<>>><><>><<<<><<<><<>><>>><><<>>><<<>>><<>><<<<>>>><<<>><>>><<<<><<>>>><>>>><<>>><<<>>>><>>><><<<>>>><><<<<><<>>><<<<>>><><<>>>><>><<<><<>><<><<<<><<<<>>>><<<<><>>>><<<>>>><<<<>><>>><<<>><<<><<<<>>><>><<<<>><<<>>>><<<>><>><<>><>><<<>>>><>>>><<>>><<><<<>><<<<><<>>><>>>><<>>>><>>>><<<<>>><<>>><<<><<<>><>>>><<>>><<<<>>>><<<<>>>><>>><<>><<<<>><>>><<<>><>>><>>>><<>><<<><<<>>><<<><<<>>>><>><<<>><>>>><<<<>>><<<<>>>><<<><>>>><>><<<<>>>><<<<><<>><<><>><<>><><<<>>>><<<<><<<<>>><<<><>><>><<<>>>><<>><<><>>>><<<<>><<><<<<>>><<<><>>>><<>>><<<<>>>><>>><<<><><<<><<<><>><<<<>><<<<>>>><<<>><<<<>>><<><>>>><<<>>><><<<>>>><>>><<>><<>>>><<<>><<<>><<<>><<<<>>><<<><>><<<>>>><<>>><>>><<<<>><<>>>><<<<>>>><<>>><<<>>><><><<>>><>>><<>>>><>><<<<>>><>>>><<<<>>>><>><<<>>><<<<>>>><<<<>>>><<<<><<<>><<<>><<<><<<<>>><<<>><>>>><>>><<<<>>><<<<>>><<>><<>><><<>><<<>><<>>><<<>>><<<>>><<<>>>><>>><<<<>>><>>><<<>>>><<<>>>><<>>><<<<>>><><<><>><<<<><<>><<>>>><<<>><<<>>><<<>><<<<>><<>>>><<>><<<>>>><><<<>>>><>>>><<><<<>>>><><<>>><<<<>>>><>>><><<<><<<>>>><<<>>><<<>>>><>>>><>>><>>>><<<>>><<<>>><><<<<>><>>>><<<>>><<<>>><<<<>><><<>>><<<>>>><<<<>>><>>><<<<>>><>><>>><<<><<<<>>>><<<>><<<<><<<<>><<>>><<<>>>><<<<>><<>><>><<<<>><<>>><<>>>><<<<>><<<>>>><><<>>>><<>>>><<>><<<>>>><<<>>>><>><<>>>><<<>>>><<>>><>><<><><>>><<><<>>><<<<>>>><<>>>><<>>>><<>>><<<<>><<>>><<<<><<<<>><<<<><<>><><<<<>>>><<<<>><<>>>><<>><<>>>><<>>><>>><<><<<><<<<>>>><<<<>><>>><<<<>>><<<>>>><<>>><<><<><>><<<>><<>><<<><<>>>><<<>>>><<<<>>>><<>><<<>>><<>>>><<<><<<<>>>><<>>><>>><<<<>>>><<<>><<><>>><<<<>><<>><>>>><<<>>><><<<>><><<>><<<>>>><<>><<><<>><<<<>>>><<>><>>>><<>><>><<<>>><<><<<>>><>>><<><<<<>><<<<>>>><<>><<<<>>>><>>>><<<>>><><>>>><<<>>><>>><<>>><<<>><<<<>><<<>>><>>><><<<<>>>><<<><<>>>><<><><<>><<>>><<>>>><>><>>><<<<><<<>>><<<<>>><<<<><>>><<<<><<<>>>><<>>><<<>>>><<<<>>><<<>>>><<>>><<<<>>>><>><<<>><<<<><>>>><>>>><>>><<<<>>>><>><<<<><>><<<>><<>>><<>>>><<<<>>><<<>>><<<>><<<>><<<<>><<<<><><<<><<><<<>><<<><><>><<>><><><<<<>>><<>>><<<<>>><>><<<<>><<<>><<>>><<<>>><>>><<<>><<>>>><<<<>>><<<>><<<><<<><<><<<>><>><<<<>>><<<<>><<>><><<<>><<<>>>><<>>>><<<><<<<>>>><<<<><<<><<<>>><<<<><<>>><><<<>>><<<>>><>>><>><>>>><<<<>>>><<>>><><<<>>>><<<>>>><<<>>>><>><<>><<<<>><<<<>>><<<<>>>><<><<<>>>><>>><<<>><<>>><<<<>>>><<<><<<<>><<<<><>><<<<>><>>><<<>>>><<<>>><<>>><<>>>><<>>><><<<>>>><<><<<>>>><<>><<<<><<<><<<><<>>>><><<<<><><>><<<>>>><<<<>><<<>><>>><<<<>>><<<<><<>>><<>>><<<<>><<<><>>><>><<<><>>><<<>>>><<<<>>><<<<><<<<>>><>>>><<<>><<<<><<<<>><<<<><<<<>><<<>>><<><<><>>>><<>>><<<><<<<><<>>>><<<<>>><<>><><<<>>>><<>><<<>>>><<><<>><<<<>>><><<<>>>><<<>><<<>><<>>>><<<>>>><<><<<<><>>><<>>><>><<>><<<<>>><>><<>>>><<<>><<<<><<<<><<>><<<<><<>>><<<>>><<<<><<<>><<<<><>><<<<>>><<<<>><<<<><<><><<<><>><><<<<>>><>>>><<<><<<<>><<>><<<<>>><<<>>><<<>><>><<<<><>><><<<>>><<><<<<>>>><>><><<>>><>><<<><<<<>>>><<>>><<<>><<<<>>><<>>><>>>><<>>>><>>><<<><>>>><<>>><><>>><<<><<<<>>>><<<<>><<<><<<>>>><<><<>>>><>><>>><<<>>>><<<<>><<<<>><<><<<>>><<><<<<>>><<<<><<<<><<<>>><<<><>>>><>>><<>>><<>>><<<<><<<<>><<<<>>>><<>>><<><<<<>>><<<<><<<<>>><<>><<<>>><<<<>><<>><<<<><>><<>>><>>><<>>><<<<>><<<<><><<<>><<>><<>>>><<>>><>>>><<<<>>>><<<<>>>><<<<><>><<<<><><<<<>>>><<<><<<<><<<<>>>><<<>><<>><>>><<<>>><>>>><<<>>>><<<<>><>><<<>><<><><<<><<<<>>><<<<><<<><>><>>><<<<>><>><<<>><<<<><<>>><<<>>><<>>>><>><><>><><<<>>>><>>><<><<<<><>>>><<><<>><>><<<>>><>>><<<<><<<>>><<<>>>><<<>>><<<>><<>>><<>>>><>><<<<>><<<<><>><<>>><<>>><<><<>>><<<<>>>><<<>>><<>><<<><<<<>><<<>><<><<<<><>>><<<>><<<<>>>><<<><<<>><<<><<<<>><<>>>><<<<>>>><<<><<<>>>><<<><<<><><<<<>>>><<<>>>><<<>>><<<><<<>><<<<>>><>>><<<<>><<<>>><<<<>>><<<<>><<><<<<><<>><<>>><<>>><<><<<<>><>>><<<>><<>>>><<<>><>>>><<<>>><>>><<><<<<>><<<<>><<<<>>><<<<>>><<<>><>>><<<<>>><<<><<<>>>><<<><<<<>><<<<>>><<<>>>><<<>>><<><><>>>><<<>><<<>>>><>>>><<<<>>>><><<<<>>><>><<<<>>><<<><>>><<<>>><<<<><<<>>><<<<>>><>>><>><<<<><<><<<<>>><>><<>>>><<>>><><>>>><<<<>>>><><<>>>><<>>>><>>>><<>>>><>><>>><<<<>>><<>><<<<>>><<<>><<<<><<>><<><<><<<><<>>><<<<>><<><<<>><<<>>>><<>>>><<><<<>>><<>><>>><<<><<<>>><<<<>>>><>>>><<><<<<>>>><<>>><>><>>><<<>>>><<<><<<<><<<<><<>><<<<>><<<>><<<>><>><>><<>><<>>>><>>>><><<<<>>><<<><<>><<<<><<>>>><<<<><>>><<<<>>>><>>><<<>><<>><><>><>>><<<>><<>>><>><>>><<>>><<<<>>>><<<<><<<<>>><<<<><<>><<<<>><<<<><<<<>>><>>>><<><>><><<<><<<>>><<<><<<>>>><<>><>><>>><<>>>><>>>><<>>>><<>><>><<<>><<>><<>><<<><<>><><><<<>><<>><<<<><>><<>>>><<<<><<<>><>>>><>><<<><<<>>>><<>>><>>><<<<>>>><>><<>><<<<>>>><<<>>><<<<>><>>><<<>>>><<<><<><><<<<><<<>>>><<<>><>><<<><><<>>>><<<<><<>>><>><>>><<<>><<>>>><<>>><<><<>>>><<>>><<<<><>>><<<>>><><>><<>>>><<<<>>><>>><><<<<><<><<<><>>>><>><<<><<>><><<>><<>>>><>>>><<>>>><>>>><<<><<<<>><<<<>><<>>><>><<<<>>><<<<><<<>><><<<<><<<<><<<<>><>>><>>>><<<<>><<<<>>>><<<<><><<>><<<>><>>><<>>><<>>>><<><><<<<>>><<<>>>><<<<><<>>>><<<<>><>>><>>>><<<>><>>>><<<<>>>><><<><<<><<<<>>>><<>><<<>>><<<<>><>>>><<<>>><<<>>>><><<<<>><<>><<<><>>><><>>>><><<>>>><<<<><<>>>><>>><<>>><<<<>>>><<<>>>><<<<>>><<<<>><<<<>>><<><><<>>>><>>>><><<>>><>>><<><<><<><<<>>><<<><<><<<>>><>>><<<>>><<<<>><<<<>><<>>><<<<>>><<<>><>><>><<><<<><<>>><<><<<><<>><>>><<<>>><>>><>>><<>>>><<<><<>>><><<<>>><>>><>><>>><>>>><<><<<><<<>>>><<<<>>>><<<>>>><<>>>><<<><>><<<<>>>><<<<><>>><><<<<><>>>><>><<<>>>><>>>><<>>><<<<>><>>>><<>>><<>>><<>>><<<>>>><>><<>>>><<<<><<<<>>><<<<>>>><<<<>>><>>><<<<><<>><>>><><<<>>>><>>><<<<>><<<><<<<><<<>>>><>>>><>>>><<<<><<>>>><<<<>><<<<>>>><>><<><<>>>><<><<<<>>><<<>>>><<<<>><>>><<<><<<>><<<><<<>>>><<>>>><>>><<<>>>><<>><<>>>><<<<>><>>>><>>><<>>><<<>>><<>>><>><<>><<>><<<>><<<<><<<>>>><>><><<<<>>><<<<>>><<>>>><><<<>>>><<>>>><><<<<><<<>><<<>>>><<<<><<<<>><<>>>><<<<>>>><>>><<<<>><>>><<><<<<>>><<<>>><>>><<<>>><><>><<<<>>><>>>><<<>><<>>>><<>>>><>>><<<><<<>>><<<<>>>><<<>>><><<<<>>>><><<<>><>><<<><<><><<<>><<<>><<<<>><<>><><>><<<>><<<>>><>><<<<>>><<<>>>><<>>>><<>>>><<<>><<<>>>><<<<><<<>>><>>>><>><>><<><<><>>>><>><<<<>><<<>>><>>>><<>>><<<>><<<<>><<<>><<<<>><<>>><<<><>><<<>><<<><<<<>>>><<>><<>>>><>>><<><<>>>><<<<><<>>><<><<<<>>><<<<>>><<<>>>><<<<><<><<<<>><<<><<<<>>>><<>><>><<<<>>><<>>>><>>>><<<<><<><<<<>>><>>>><<<><<>>><<<<><<<>>><<<>><<<<>><>>><>><<>>>><<<<>>>><>>><<><<<>><<>>><>><<<<>>><<<>>>><<>><<<>>>><<<<>><<<>>>><>><><<<<>>><<<>>>><<>>>><<<<>><<<<>>>><><<<<>><>>><<<<><<<>>><>>><<>>><><<><<>><<><<>>>><<<>>>><<<<>><<<<>>><>>><<<><<>>>><<><<>>><<<<><<>><<<<>>><<<<>><<<<>>>><>>>><><<<<>><>><>>>><<>>><<<<>>><<<><<<<>>><><<>>>><<>><<<>>>><<<><>>><><<<<><<<><<<>><<>>>><<>>><<<<>><><<<>><<>>><>>>><<<>><<>>><<>>><<<<>>>><<<<>>>><<>>>><<<<>><<<>>>><<><<><><<<>>><<<<>>><>>>><<>><<<>><<<<><<>><<>><<>><>>>><<<<>>><<<<>>><>>><>>>><>>><>>><<<><<<>>><<<>><>><<>>>><<<>>><<<>><<<>><<>>><<<>>>><<<><<>>><<><<<>>><<<>>>><>>><<<>><<><<<<>><<<>><<<<><<>><<<<>>>><>>>><<<<><<<<>>>><<<>><<<<><<<>>>><<<><<<<>>><<<><>>><<>>>><>>>><<><<>><<>>><<<<>>><<<<>>>><<<>>>><<<<>>>><>><<<>>><><<>><>>>><><<><<<<>>><<>>>><>><<<>>><><<<><<>><<<>>>><<<<>>>><<<<>>><<<<><<<<>><>>>><<<<>><<<<>><><<<<>><<<>><<>>>><>>><<<>>><>>>><<<>><<<>>><<<>><<<<>>><<<<><<>><<>>>><<<>><>>>><<<<>>><<<>><<<>>><<<<>>><><>><<><>>><><><>><<>>><<<><<<<>>>><<<>>>><<<>>><<>><<>>>><<<<><<<<>>>><<<<>>><<<>><<<<>>><<<>>><>>>><<><>>><<<<>>>><<<>>>><<<>><>><>>><<<<><<<>>>><<<<>>>><>><<<<><<<><<<><<>>><<>><<<><<>><<>>><<<>><<<<><<>>><<<><>>><>>><<<>>><<>>>><<<<><<<<>><<>>>><<<<>><<<>>>><<<>>><<><<><<><<<>><<<<>><>>><><<<<><<<>>>><<><<><<<>><<<<>>><<>><<<<>><<<<><<<>>>><<>><<><<<<>><<<>>>><<<>>><>><>><><>>>><<<<>>><>>><<<<>><<<>>>><<<>>><<<>>><<<><<<>>>><>>>><<<<>>>><>>>><>><<<<><<<<>>>><<<<>><<<<>><<<<><<<>><<>><<><<<<>>><<<>><<>>>><<>>><>>>><<><<<<>><<<>>>><<>>>><>>>><<><<>><>>>><<>><<<<>>>><>>>><>>>><><<>><<<<>>>><><>>>><<<>><<<<>><<<>>><>>>><<<><<<><<>>>><<<>><<<>>>><<<>>><>>>><><<>>>><<<>><<<>>><<>>><>>><<<>>><<<>>><<<><<><<<>>>><<<>><<<>>><<<<>>><<<>>><<<><><<<>><>>><><<<<>><<<<>>>><<<>>>><>>>><<<<>><><<<<>>>><>>><<<<>><<<<>>><<<<>>><<><<<><>>><<<>>>><<<>><<>><<><<<><<>>>><<<>>>><<<>>><<<><<<>>>><<<>><<<<>>>><>><<<<><<<<><>>><>>><<><<><<>>>><<<><>>><><<<<><<<>><<<>><<<>>><<>>><<<<>>><<>>><>>><<<>>><>>>><<><>>><<<>>>><<<>>>><>>><>>><>><>>><<<<>>>><<<><<<>><><<>><<<><<<<><<<>>>><<>>><>>>><<<<>><>>><<<>>>><<>>><<<>><<<><>><<<>>><<>>>><<<<>>>><<><<>><<><<<><<>>><><<<<>><<>>>><>>>><>>>><<><>><<<>>><<<><<<>>>><<<<><<>>><<<<><>>>><<<><<<>><<><<<<>><<><<><<>>>><<<<>>>><<<<><<>>>><>>>><<<<>>>><<>><<><<<>>>><><<<<>>>><<>>><<<><<<<>>><<<<><<>>><>>><<><<<<>>><<<>>><<<>>><<<<>>><<>>>><<<><<<>>>><<>><>>><<>>>><<<>><<<><<<<>><<>><<<<>>><<<<><<<<><<<>>><>><<<<>><<>><>><>><<<>>><<>><<<<>>><>>>><>>>><<>><<<<><<<<>>>><><<<><<<><<>><<<<>><<<<>>>><><<>><<<<>><<<<>><<<<>>>><>>><<<<>><<<>>><<<>><<<<><<<<><<<<>><><<<<>>><>>><<<<>><<>>><<>>>><<<>><<>><<<<>><>><>><<<<>>><<<>><<>>>><<<>><<<<>>><<><><<<<><<<>>>><<<>>>><<<<><<>>><><>><<<<>>><<<<>>><<>><<<<>><>>>><<<>><>>>><>><<<<>>>><<<<><>><<<<>><<<>><><<<>>>><<><<<<>><<>>>><<<><<<<>><<<>>><<><<><<<>>>><<<><<>><<>>>><<<>><<<>>><<<>>><<<>>><<<<><><<<><>><<<<><<<<>>>><<>>><>>>><<<><><<<><<<>>><<<>><><<<>>><>>>><<<>>><>>>><><<>><><<<<><<>>>><>><>>>><<>><><<<>><<>>>><><<><<<<>>>><<<<>>>><<<>><<<>><<>>>><<<>>>><<<>>><><<>>><<<<><><<<<>>>><<>><<>>>><>>><<><<>>>><>>><<>>>><>><>>><>><<<>>>><>>>><<><<<<>>><>>>><<<><<>>><<<>>>><<<>><<<<>><<>>><<<><>>>><<<<>><<>>><<>><><<><<<<><<<>>><>><>>><<<<><<<<>>>><<<>>><<<>><<<<>><<<>>>><>>>><<<<>>><><<>><<<>>>><<<<><<>>><<>><<>>><<<<>>><>>><<<<>>><<<<>>><<<<>><<>>>><<<<><>>><<<<>>>><<<<>>>><<<<>>><<<>><<<><<<>>>><<>>>><<<><<<<>>><<<<>>>><>><<<><>><<>>><<<><<>>>><>>><<<<>>>><<>><<><<<<><<<<>>><<<<>>>><<>>>><<<<>>><<<<>'
rounds = 2022
g = Grid(input)
# g.add_rows_to_top(10)
for _ in tqdm(range(rounds)):
    g.drop()
    # print(g)
 

g.height()
# g.grid[:10]
# g

100%|██████████| 2022/2022 [00:00<00:00, 2912.75it/s]


3188

In [116]:
g = Grid(input)
g.run(rounds)

i=2022


3188

# Second part

In [117]:
rounds = 1000000000000

g = Grid(input)
g.run(rounds)
 


skipped 999999997245 iterations
i=1000000000000


1591977077342